# make_real_polygons

2022/1/4

real_polygonsを生成するためのノートブック

以下のような辞書を、各カテゴリ毎に作る。

```
{
    key: [[x1, y1, x2, x2, ...], [x1, y1, x2, y2, ...]],
}
```

In [1]:
import os
import pickle
import json
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab

In [10]:
VAL_IMAGES_DIRECTORY = "/home/nakachi/data/stair/val2014"
INSTA_VAL_ANNOTATIONS_PATH = "/home/nakachi/data/coco2014/annotations/instances_val2014.json"
insta_val_coco = COCO(INSTA_VAL_ANNOTATIONS_PATH)

loading annotations into memory...
Done (t=5.19s)
creating index...
index created!


In [11]:
'''
保存するpickleのディレクトリ指定
'''
POLYGONS_DIR = '/home/nakachi/data/real_polygons/'

In [ ]:
'''
category毎にpickleを作る
ex. cat_polygons.pickle
'''
for cat in insta_val_coco.cats.values():
    # categoryのループ
    cat_id = cat['id']
    cat_name = cat['name']
    print(f'process {cat_name}...')

    # polygons = segmentation
    real_polygons = dict()
    for img in insta_val_coco.imgs.values():
        # 画像のループ
        img_id = img['id']
        img_key = img['file_name'].replace('.jpg', '')
        ann_ids = insta_val_coco.getAnnIds(imgIds=[img_id], catIds=[cat_id], iscrowd=None)
        annotations = insta_val_coco.loadAnns(ann_ids)
        
        segmentations =[]
        for ann in annotations:
            if type(ann['segmentation']) == list:
                # polygonなので追加
                segmentations = segmentations + ann['segmentation']
            else:
                # mask????いらない
                pass

        real_polygons[img_key] = segmentations
    
    # 保存
    save_pickle_path = os.path.join(POLYGONS_DIR, cat_name + '.pickle')
    with open(save_pickle_path, mode="wb") as f:
        pickle.dump(real_polygons, f)
    print('----------------------')

---
作ったもの確かめる

In [13]:
real_polygons_path = os.path.join(POLYGONS_DIR, 'cat.pickle')
with open(real_polygons_path, 'rb') as f:
    f_pickle = pickle.load(f)

In [15]:
len(f_pickle)

40504

---
OKそう。

ただpolygonではなく、maskの時はスキップしている処理があるのだが、それでいいのか懸念。

どれぐらいマスクあるのか調べる。

In [ ]:
'''
category毎のマスクが
どれぐらいあるのか調べる
'''
for cat in insta_val_coco.cats.values():
    # categoryのループ
    cat_id = cat['id']
    cat_name = cat['name']
    print(f'process {cat_name}...')

    polygon_cnt = 0
    mask_cnt = 0
    for img in insta_val_coco.imgs.values():
        # 画像のループ
        img_id = img['id']
        img_key = img['file_name'].replace('.jpg', '')
        ann_ids = insta_val_coco.getAnnIds(imgIds=[img_id], catIds=[cat_id], iscrowd=None)
        annotations = insta_val_coco.loadAnns(ann_ids)
        
        segmentations =[]
        for ann in annotations:
            if type(ann['segmentation']) == list:
                # polygonなので追加
                polygon_cnt += 1
            else:
                mask_cnt += 1
    print(f'polygon cnt = {polygon_cnt}')
    print(f'mask cnt = {mask_cnt}')
    print('----------------------')

結構少ないので無視すると判断

---

結局、スケールをあわしたので保存した方がいいので再度作る。

最初は `utils.py` の `build_super_images` 関数のために
272サイズでやる。

In [2]:
VAL_IMAGES_DIRECTORY = "/home/nakachi/data/stair/val2014"
INSTA_VAL_ANNOTATIONS_PATH = "/home/nakachi/data/coco2014/annotations/instances_val2014.json"
insta_val_coco = COCO(INSTA_VAL_ANNOTATIONS_PATH)

loading annotations into memory...
Done (t=5.17s)
creating index...
index created!


In [3]:
insta_val_coco.cats

{1: {'supercategory': 'person', 'id': 1, 'name': 'person'},
 2: {'supercategory': 'vehicle', 'id': 2, 'name': 'bicycle'},
 3: {'supercategory': 'vehicle', 'id': 3, 'name': 'car'},
 4: {'supercategory': 'vehicle', 'id': 4, 'name': 'motorcycle'},
 5: {'supercategory': 'vehicle', 'id': 5, 'name': 'airplane'},
 6: {'supercategory': 'vehicle', 'id': 6, 'name': 'bus'},
 7: {'supercategory': 'vehicle', 'id': 7, 'name': 'train'},
 8: {'supercategory': 'vehicle', 'id': 8, 'name': 'truck'},
 9: {'supercategory': 'vehicle', 'id': 9, 'name': 'boat'},
 10: {'supercategory': 'outdoor', 'id': 10, 'name': 'traffic light'},
 11: {'supercategory': 'outdoor', 'id': 11, 'name': 'fire hydrant'},
 13: {'supercategory': 'outdoor', 'id': 13, 'name': 'stop sign'},
 14: {'supercategory': 'outdoor', 'id': 14, 'name': 'parking meter'},
 15: {'supercategory': 'outdoor', 'id': 15, 'name': 'bench'},
 16: {'supercategory': 'animal', 'id': 16, 'name': 'bird'},
 17: {'supercategory': 'animal', 'id': 17, 'name': 'cat'},

In [4]:
'''
保存するpickleのディレクトリ指定
'''
POLYGONS_DIR = '/home/nakachi/data/real_polygons_rescale_272/'

In [13]:
import skimage.transform
def polygon_resize(old_width, old_height, segmentation, width, height):
    # 圧縮する比率(rate)を計算
    r_width = width / old_width
    r_height = width / old_height

    # 比率を使ってBoundingBoxの座標を修正
    new_segmentation = []
    for seg in segmentation:
        new_seg = []
        for i in range(0, len(seg), 2):
            new_seg.append(seg[i] * r_width)
            new_seg.append(seg[i+1] * r_height)
        new_segmentation.append(new_seg)
    return new_segmentation

In [14]:
'''
category毎にpickleを作る
ex. cat_polygons.pickle
'''
IMG_SIZE = 272

for cat in insta_val_coco.cats.values():
    # categoryのループ
    cat_id = cat['id']
    cat_name = cat['name']
    print(f'process {cat_name}...')

    # polygons = segmentation
    real_polygons = dict()
    for img in insta_val_coco.imgs.values():
        # 画像のループ
        img_id = img['id']
        file_name = img['file_name']
        img_key = file_name.replace('.jpg', '')
        img_width = img['width']
        img_height = img['height']
        
        ann_ids = insta_val_coco.getAnnIds(imgIds=[img_id], catIds=[cat_id], iscrowd=None)
        annotations = insta_val_coco.loadAnns(ann_ids)
        
        segmentations =[]
        for ann in annotations:
            if type(ann['segmentation']) == list:
                # polygonなので、resizeして追加
                new_segmentation = polygon_resize(img_width, img_height, ann["segmentation"], IMG_SIZE, IMG_SIZE)

                segmentations = segmentations + new_segmentation
            else:
                # mask????いらない
                pass

        real_polygons[img_key] = segmentations
    
    # 保存
    save_pickle_path = os.path.join(POLYGONS_DIR, cat_name + '.pickle')
    with open(save_pickle_path, mode="wb") as f:
        pickle.dump(real_polygons, f)
    print('----------------------')

process person...
----------------------
process bicycle...
----------------------
process car...
----------------------
process motorcycle...
----------------------
process airplane...
----------------------
process bus...
----------------------
process train...
----------------------
process truck...
----------------------
process boat...
----------------------
process traffic light...
----------------------
process fire hydrant...
----------------------
process stop sign...
----------------------
process parking meter...
----------------------
process bench...
----------------------
process bird...
----------------------
process cat...
----------------------
process dog...
----------------------
process horse...
----------------------
process sheep...
----------------------
process cow...
----------------------
process elephant...
----------------------
process bear...
----------------------
process zebra...
----------------------
process giraffe...
----------------------
process ba

In [15]:
real_polygons_path = os.path.join(POLYGONS_DIR, 'cat.pickle')
with open(real_polygons_path, 'rb') as f:
    f_pickle = pickle.load(f)

In [ ]:
f_pickle